In [1]:
%load_ext autoreload
%autoreload 2

## Running experiments externally

Running experiments externally is the most common use of summit. There is a simple set of commands that allows you to do this. Let's say we are going to run the SnarBenchmark ourselves instead of in a closed-loop setup. We'll use TSEMO since this is a multiobjective problem.

In [22]:
# Import the necessary classes
from summit.benchmarks import SnarBenchmark
from summit.strategies import TSEMO2
from summit.utils.dataset import DataSet

In [23]:
# We are just going to get the domain from the SnarBenchmark
lab = SnarBenchmark()

# Instatiate TSEMO
tsemo = TSEMO2(lab.domain)

# To get the first experiments
next_experiments = tsemo.suggest_experiments(5)
next_experiments

,tau,equiv_pldn,conc_dfnb,temperature,strategy
0,1.55,2.2,0.22,57.0,LHS
1,1.25,4.6,0.30,39.0,LHS
2,0.65,3.0,0.38,111.0,LHS
3,0.95,3.8,0.46,93.0,LHS
4,1.85,1.4,0.14,75.0,LHS


You'll notice that the first set of experiments was generated by LHS or latin hypercube sampling. We'll need to go to the lab and run experiments at those conditions. 

Since it might be a while before we come back, let's save what we've done up until now. We save a file called `snar_tsemo_external.json` which contains all of the information we need to reproduce our work up to this point.  We will also produce a CSV with the suggested conditions, which we can also use to record the results of our experiments. 

In [ ]:
tsemo.save('assets/snar_tsemo_external.json')
tsemo.reset()
next_experiments.to_csv('assets/snar_experiments_external_0.csv')

Once we have run the conditions, we can go ahead and load the data back in.  For convenience we've created a file called `snar_experiments_external_1.csv`, which already has some data added.

In [ ]:
tsemo = TSEMO.load('assets/snar_tsemo_external.json')
prev_res = DataSet.read_csv('assets/snar_experiments_external_1.csv')

Now, we can run TSEMO again and ask for new suggested conditions based on the first set of experiments.

In [ ]:
next_experiments = tsemo.suggest_experiments(5, prev_res=prev_res)
prev_res.append(next_experiments)
prev_res.to_csv('assets/snar_experiments_external.csv')
tsemo.save('assets/snar_tsemo_external.json')
next_experiments

We can repeat the the cycle of saving and loading the tsemo paramters and next experiments until we are satisfied with the results.

## Running Closed Loop Experiments

Summit has speciial functionality for running closed-loop experiments.  The runner class takes a strategy and experiment. It then get experiment suggestions from the strategy and runs the experiments.

Note that in order to see the progress bar in jupyter, you'll need to install the jupyter or jupyterlab extension for ipywidgets. See [here](https://ipywidgets.readthedocs.io/en/latest/user_install.html#installing-the-jupyterlab-extension) for details.

In [3]:
# Import the necessary classes
from summit.benchmarks import Hartmann3D
from summit.strategies import SNOBFIT
from summit.utils.dataset import DataSet
from summit.run import Runner

In [67]:
# Establish the experiment. Here, we are using one of the single objective test functions
exp = Hartmann3D(maximize=True, constraints=False)

# Set up the strategy, passing in the experimental domain.
snobfit = SNOBFIT(exp.domain)

# Use the runner to run closed loop experiments
r = Runner(strategy=snobfit, experiment=exp, 
           max_iterations=5)
r.run()

In [16]:
import numpy as np
s = snobfit.to_dict()
snobfit_2 = SNOBFIT.from_dict(s)

In [ ]:
snobfit.save('snobfit.json')